# XLNet version 1

Hate speech identification project, D7047E <br>
Binary text classification task using pretrained XLNet models

In [54]:
""" 
%pip install torch
%pip install torchvision
%pip install sentencepiece
%pip install transformers
"""

import torch
import torch.nn as nn
import torch.nn.functional as F

import torchvision

from transformers import XLNetModel, XLNetConfig, XLNetTokenizer, AutoTokenizer, XLNetForSequenceClassification

from tqdm import tqdm
import sentencepiece as spm
import os

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f"using {device}")

using cpu


In [75]:
# Constants and vars

# Note: all below are .tsv format 
path_tr = r"..\\OLID_Tain_ATUSER_URL_EmojiRemoved_Pedro.txt" 
path_te_a = r"..\\OLID_TEST_A_ATUSER_URL_EmojiRemoved_Pedro.txt" 
path_te_b = r"..\\OLID_TEST_B_ATUSER_URL_EmojiRemoved_Pedro.txt" 
path_te_c = r"..\\OLID_Tain_ATUSER_URL_EmojiRemoved_Pedro.txt" 
path_temp_vocab_tsv = r"data\\temp_vocab_tsv.txt"

# Model configs
GLOBAL_VOCAB_SIZE = 20_000
ALWAYS_OVERWRITE_VOCAB = True
config = XLNetConfig()

In [80]:
# Load data, format, preprocess
#tokenizer = AutoTokenizer.from_pretrained("xlnet/xlnet-base-cased")

# create intermediate 

if ALWAYS_OVERWRITE_VOCAB or not os.path.isfile(path_temp_vocab_tsv):
    print("Creating intermediate vocab help file")
    res_s = ""
    with open(path_tr, 'r') as trf:
        for i, line in enumerate(trf.readlines()):
            if i > 0:
                l = line.split("\t")[1:2]
                res_s += l[0]+" "

                if i > 10: break
    
    with open(path_temp_vocab_tsv, 'w') as tmpf:
        tmpf.write(res_s)
else:
    print("File already found")


spm_tr = spm.SentencePieceTrainer.train(
    input = path_tr,
    model_prefix='en-sp', 
    input_format = "bsp",
    vocab_size = GLOBAL_VOCAB_SIZE
)

Creating intermediate vocab help file


RuntimeError: Internal: D:\a\sentencepiece\sentencepiece\src\trainer_interface.cc(329) [trainer_spec_.input_format().empty() || trainer_spec_.input_format() == "text" || trainer_spec_.input_format() == "tsv"] Supported formats are 'text' and 'tsv'.

In [39]:
# XLNet model
# docs: https://huggingface.co/docs/transformers/model_doc/xlnet
model = XLNetForSequenceClassification.from_pretrained("xlnet/xlnet-base-cased")
print(model.config)

Some weights of XLNetForSequenceClassification were not initialized from the model checkpoint at xlnet/xlnet-base-cased and are newly initialized: ['logits_proj.bias', 'logits_proj.weight', 'sequence_summary.summary.bias', 'sequence_summary.summary.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


XLNetConfig {
  "_name_or_path": "xlnet/xlnet-base-cased",
  "architectures": [
    "XLNetLMHeadModel"
  ],
  "attn_type": "bi",
  "bi_data": false,
  "bos_token_id": 1,
  "clamp_len": -1,
  "d_head": 64,
  "d_inner": 3072,
  "d_model": 768,
  "dropout": 0.1,
  "end_n_top": 5,
  "eos_token_id": 2,
  "ff_activation": "gelu",
  "initializer_range": 0.02,
  "layer_norm_eps": 1e-12,
  "mem_len": null,
  "model_type": "xlnet",
  "n_head": 12,
  "n_layer": 12,
  "pad_token_id": 5,
  "reuse_len": null,
  "same_length": false,
  "start_n_top": 5,
  "summary_activation": "tanh",
  "summary_last_dropout": 0.1,
  "summary_type": "last",
  "summary_use_proj": true,
  "task_specific_params": {
    "text-generation": {
      "do_sample": true,
      "max_length": 250
    }
  },
  "transformers_version": "4.40.1",
  "untie_r": true,
  "use_mems_eval": true,
  "use_mems_train": false,
  "vocab_size": 32000
}



In [40]:
# Run + testing
inputs = tokenizer("Hello, my dog is cute", return_tensors="pt")
print(inputs)
outputs = model(**inputs)
print(type(outputs))

{'input_ids': tensor([[   17, 11368,    19,    94,  2288,    27, 10920,     4,     3]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 2]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1]])}
<class 'transformers.models.xlnet.modeling_xlnet.XLNetForSequenceClassificationOutput'>


In [43]:
print(outputs.logits)

tensor([[0.3253, 1.2618]], grad_fn=<AddmmBackward0>)
